Topics in the Notebook: 
- Read the list of images sing queues and coordinators
- Resize the images to the same dimensions
- show the images in TensorBoard

In [1]:
import tensorflow as tf
from PIL import Image

C:\Users\ku.kulshrestha\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ku.kulshrestha\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ku.kulshrestha\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint

In [2]:
original_image_list = ['droplets.jpg', 'leaf.jpg', 'night_boat.jpg', 'tiger.jpg']

In [3]:
#make a queue of file names of images
filename_queue = tf.train.string_input_producer(original_image_list)

In [4]:
#Read the file 
image_reader = tf.WholeFileReader()

In [5]:
with tf.Session() as sess: 
    #Coordinate the loading of image files
    coord = tf.train.Coordinator()   #It abstracts away all queue working in multiple threads
    threads = tf.train.start_queue_runners(sess = sess, coord = coord) #starting all of the threads
    
    image_list = []
    for i in range(len(original_image_list)): 
        #reading the whole file from the queue
        _, image_file = image_reader.read(filename_queue)
        #decode the jpeg image and convert it into a tensor
        image = tf.image.decode_jpeg(image_file)
        #resizing the image
        image = tf.image.resize_images(image, [224, 224])
        image.set_shape((224, 224, 3))
        
        #execute the computation graph
        image_array = sess.run(image)
        print(image_array.shape)
        
        #Displaying the resized image
        Image.fromarray(image_array.astype('uint8'), 'RGB').show()
        
        #We want every image to be represented by a single 4 dimensional tensor 
        #with first element reflecting which image number it is.
        image_list.append(tf.expand_dims(image_array, 0))
    
    coord.request_stop()
    coord.join(threads)
    
    #Representing the image in tnsorboard
    index = 0
    
    #Write image summary
    summary_writer = tf.summary.FileWriter('./m4_example2', graph = sess.graph)
    
    for image_tensor in image_list: 
        summary_str = sess.run(tf.summary.image("image-" + str(index), image_tensor))
        summary_writer.add_summary(summary_str)
        index +=1 
    
    summary_writer.close() 
        
        

(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)


### List of images as 4D tensors with Image transformations

If we have to represent all images in the same 4D tensor, all of them need to be of same size.

- (10, 6, 6, 3) --> Here 3 represents the channels, and 10 represents the total images and 6x6 being the size of each one

In [6]:
import tensorflow as tf
from PIL import Image

original_image_list = ['droplets.jpg', 'leaf.jpg', 'night_boat.jpg', 'tiger.jpg']

#make a queue of file names of images
filename_queue = tf.train.string_input_producer(original_image_list)

#Read the file 
image_reader = tf.WholeFileReader()

In [7]:
with tf.Session() as sess: 
    #Coordinate the loading of image files
    coord = tf.train.Coordinator()   #It abstracts away all queue working in multiple threads
    threads = tf.train.start_queue_runners(sess = sess, coord = coord) #starting all of the threads
    
    image_list = []
    for i in range(len(original_image_list)): 
        #reading the whole file from the queue
        _, image_file = image_reader.read(filename_queue)
        #decode the jpeg image and convert it into a tensor
        image = tf.image.decode_jpeg(image_file)
        #resizing the image
        image = tf.image.resize_images(image, [224, 224])
        image.set_shape((224, 224, 3))
        
        #To flip the image up and down
        image = tf.image.flip_up_down(image)
        
        #image cropped from center
        image = tf.image.central_crop(image, central_fraction=0.5)
        
        #execute the computation graph
        image_array = sess.run(image)
        print(image_array.shape)

        #image array is a numpy array and we want to convert it to a tensor
        image_tensor = tf.stack(image_array)
        
        #We want every image to be represented by a single 4 dimensional tensor 
        #with first element reflecting which image number it is.
        image_list.append(image_tensor)
    
    coord.request_stop()
    coord.join(threads)

    #converts all tensors to a single tensor of 4 dimension:
    image_tensor = tf.stack(image_list)
    print(image_tensor)
    
    
    summary_writer = tf.summary.FileWriter('./m4_example3', graph = sess.graph)
    
    summary_str = sess.run(tf.summary.image("images", image_tensor, max_outputs=4))
    summary_writer.add_summary(summary_str)
    
    summary_writer.close()

(112, 112, 3)
(112, 112, 3)
(112, 112, 3)
(112, 112, 3)
Tensor("stack_12:0", shape=(4, 112, 112, 3), dtype=float32)
